In [1]:
from numpy import *
import numpy as np
import healpy as hp
import time
import astropy
from astropy.time import Time
from astropy.coordinates import SkyCoord

In [ ]:
dir='~/'

In [2]:
def rot_map(basemap,ra_rot):
    """Function takes in two arguments, an initial map and a rotation matrix"""
    nside=hp.get_nside(basemap) #get pixel size from basemap
    rotmap=zeros(hp.nside2npix(nside)) #creates empty map of nside pixels
    theta,phi=hp.pix2ang(nside,np.arange(hp.nside2npix(nside))) #transform pixel to angle. takes nside and the pixel indices of a given nside 
    phi=phi+ra_rot #rotates current value of right ascension (phi) by ra_rot
    rotmap[hp.ang2pix(nside,theta,phi)]=basemap #creates new rotated map with rotated values for right ascension 
    return rotmap

In [3]:
def vampire(time,nside,dead_zone):
    """This function blocks the sun in plot"""
    t = Time(time, format='mjd', scale='utc')
    sun=astropy.coordinates.get_sun(t) #get coordinates of sun at specific time
    theta_sun=pi/2.-sun.dec.radian #get declination in radians
    phi_sun=sun.ra.radian #get right ascension in radians
    v_sun=array([sin(theta_sun)*cos(phi_sun),sin(theta_sun)*sin(phi_sun),cos(theta_sun)])
    theta,phi=hp.pix2ang(nside,arange(hp.nside2npix(nside)))
    v_pix=array([sin(theta)*cos(phi),sin(theta)*sin(phi),cos(theta)])
    shield=dot(v_sun,v_pix)<=cos(dead_zone/180.*pi)
    return shield

In [4]:
def bias_factor(hr,amplitude):
    """This function introduces daily cycle nonuniformitivity"""
    return  (1. + amplitude*sin(2.*pi*hr/24. - pi/8.))/24.

In [5]:
basemap_cbc=hp.fitsfunc.read_map('../BNS-MergerRates/skyprior_cbc_hl.fits') #read map
basemap_cbc=basemap_cbc/sum(basemap_cbc) #Normalize map (divide each pixel by the sum of all)

NSIDE = 128
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
Ordering converted to RING


In [6]:
mjd,ra_rot=loadtxt('../BNS-MergerRates/2017may_rot_fine.txt',unpack=True) #this file was created from the genrot_coord program

In [7]:
bias_amp=array([0.4]) #amplitude of bias = 0.4 ... will be used in bias_factor function
nside=hp.get_nside(basemap_cbc) #get nside of basemap

In [8]:
n_coadd=0
inst_map_cbc_he=zeros((hp.nside2npix(nside),size(bias_amp))) #create empty array, nside x 1

In [9]:
for i in range(0,size(ra_rot)): #for each element of ra_rot array
    rot_map_cbc=rot_map(basemap_cbc,ra_rot[i]) #rotate basemap map by that element
    rot_bias_cbc=rot_map_cbc[:,newaxis]*bias_factor((i*0.25)%24,bias_amp) #apply bias factor with bias amplitude=0.4
    shield_he=vampire(mjd[i],nside,18.)	#decide how many degrees around the sun shall be blocked.
    inst_map_cbc_he=inst_map_cbc_he+shield_he[:,newaxis]*rot_bias_cbc #to empty map, add the sun*rotated map(with bias applied)
    n_coadd=n_coadd+1 #iteration
coaddmap_cbc_he=inst_map_cbc_he/n_coadd #average out map by number of iterations

In [10]:
#not sure what this does
for j in range(0,size(bias_amp)):
    fileinfo='hl_2017may_sin'+'%.3f'%bias_amp[j]+'_0.125pi_fine'
    #write map and save
hp.fitsfunc.write_map('../BNS-MergerRates/skyprior_cbc_'+fileinfo+'_twilight.fits',coaddmap_cbc_he[:,j]/sum(coaddmap_cbc_he[:,j]))